In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import dense_correspondence_manipulation.utils.utils as utils

utils.add_dense_correspondence_to_python_path()
from dense_correspondence.dataset.dynamic_spartan_dataset import DynamicSpartanDataset
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
import dense_correspondence.correspondence_tools.correspondence_finder as correspondence_finder
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

import os
import torch
import numpy as np
%matplotlib inline

In [ ]:
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

logging_dir = "/home/peteflo/data/pdc/trained_models/imitation"
name = "sugar_contrastive_0710_3"
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
print model_folder
print os.listdir(model_folder)

if False:
    DCE = DenseCorrespondenceEvaluation
    num_image_pairs = 100
    DCE.run_detection_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)      

In [ ]:
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluationPlotter as DCEP
import matplotlib.pyplot as plt

path_to_df_csv = os.path.join(model_folder, "analysis", "detection_train", "data.csv")

fig_axes = DCEP.run_on_single_detection_dataframe(path_to_df_csv, label=None, save=True, previous_fig_axes=None)

# for network_name in nets_to_plot[1:]:
#     path_to_csv = os.path.join("/home/peteflo/spartan/data_volume/pdc/trained_models", network_name, "analysis/train/data.csv")
#     fig_axes = DCEP.run_on_single_dataframe(path_to_csv, label=network_name, previous_fig_axes=fig_axes, save=False)

_, axes = fig_axes
# axes[0].set_title("Test Set")
plt.show()

In [ ]:
import pandas as pd
df = pd.read_csv(path_to_df_csv, index_col=0, parse_dates=True)

print len(df), "is total entries"

true_positives_only = df.loc[df['true_positive'] == True]
true_negatives_only = df.loc[df['true_positive'] == False]

print len(true_positives_only), "is true positives"
print len(true_negatives_only), "is true negatives"

print min(df["best_match_diff"]), "is min best_match_diff"
print max(df["best_match_diff"]), "is max best_match_diff"

#print true_positives_only.sort_values(by=['best_match_diff'])
#print true_negatives_only.sort_values(by=['best_match_diff'])
sorted_by_diff = df.sort_values(by=['best_match_diff'])

for index, row in sorted_by_diff.iterrows():
    print row['best_match_diff']
    

In [ ]:
print len(true_positives_only['true_positive'])

In [ ]:
import seaborn as sns
import matplotlib
sns.set(style="whitegrid")
ax = sns.violinplot(x=df["true_positive"], y=df["best_match_diff"])
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 4)
plt.show()

In [ ]:
print (true_positives_only['best_match_diff'] < 3.0).sum()

In [ ]:
def compute_precision_recall(true_positives_only, true_negatives_only, best_match_diff_threshold):
    num_correct = (true_positives_only['best_match_diff'] < best_match_diff_threshold).sum()
    num_not_correct = (true_negatives_only['best_match_diff'] < best_match_diff_threshold).sum()
    precision = num_correct*1.0/(num_correct+num_not_correct)

    recall = num_correct*1.0/(len(true_positives_only))
    
    return precision, recall

ps = []
rs = []

import numpy as np
diff_min = min(df["best_match_diff"])
diff_max = max(df["best_match_diff"])
thresholds = np.arange(diff_min, diff_max, (diff_max-diff_min)/1000.0)

for threshold in thresholds:
    p, r = compute_precision_recall(true_positives_only, true_negatives_only, threshold)
    ps.append(p)
    rs.append(r)
    
ps = np.asarray(ps)
rs = np.asarray(rs)

plt.plot(rs,ps)
plt.ylim(0.0, 1.0)
plt.xlabel("recall")
plt.ylabel("precision")
plt.show()




In [ ]:
# from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

# logging_dir = "/home/peteflo/data/pdc/trained_models/imitation"
# name = "sugar_contrastive_0710_3"
# model_folder = os.path.join(logging_dir, name)
# model_folder = utils.convert_to_absolute_path(model_folder)
# print model_folder
# print os.listdir(model_folder)

# if True:
#     DCE = DenseCorrespondenceEvaluation
#     num_image_pairs = 100
#     DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)      